Credits:
- https://www.youtube.com/watch?v=N6Su4Hk8_-g (PyPDF2)
- https://www.youtube.com/watch?v=MjY7UUSAqqE&t=298s (pdfplumber)

In [1]:
import PyPDF2 # For numPages
import requests
import pdfplumber
from datetime import date

In [2]:
file = open("../../sampleFinancialStatements/sample2.pdf", "rb")
pdf = pdfplumber.open(file)
reader = PyPDF2.PdfFileReader(file)
numPages = reader.numPages
print("There are {} pages in this pdf".format(numPages))

There are 93 pages in this pdf


### Function to calculate the change in percentage %

In [3]:
# Return the change in percentage %

def calculateChange(oldVal, newVal):
    return (newVal - oldVal) / oldVal * 100

### Function to convert a string with brackets and commas to a float

In [4]:
# Return float

def becomeNumber(string):
    thingsToReplace = [',', '(', ')', '-', ' ']
    
    for i in thingsToReplace:
        string = string.replace(i, '')
        
    return float(string)

Idea is to:
- Convert the pdf page into string for further processing
- From the string, we identify the keywords
- Get the values associated with the keywords (mindful of the years, whether the new comes first or the old)
- Calculate the change and %
- Come out with the MD&A Draft text

In [5]:
keyWord = input("Please enter the keyword to look for (Not caps-sensitive) ").lower()

Please enter the keyword to look for (Not caps-sensitive) short-term


Still need to do:
- Debug the error
- If mulitple rows starts with the same keyWord, then it would be wrong
- Following example is assuming oldVal is the most right value, and the newVal is the one left of it, possible like page 9 of sample1.pdf, where there are 4 values instead
    - Likewise for the year detection in the heading
- Able to decide the time period based on the heading
    - To see which values come first
    - And also the criteria of "three period-to-period comparisons"
    - If the headings are in different pages, then it wouldn't work
        - Search the page before for the headings
- Able to get the 'units' of the money in the financial statements (is it in millions?)
    - In the sample1.pdf case, it would be '000 for staff costs
- Get the reasons of the change
- If for example is transportation expenses, do we need to auto find expenses such as "bus", "grab", etc?
- To be able to get out all the rows available by checking whether got valid value anot
- See whether can detect the currency anot
- Try for the different format as well
- Include the increase/decrease in sum as well in the % change

In [6]:
# Variables to keep track how to determine left values are the new one or the old one

found = 0     # If found = 1, means found the keyWord in PDF, if found = 0, means cannot find keyWord in PDF, need key in a new keyWord again
sameMonth = 1     # If sameMonth = 1, means need to compare by years for the time period
sameYear = 1     # If sameYear = 1, means need to compare by months for the time period
rightNew = 0     # rightNew = 1 means right value is the NEW one, rightNew = -1 means right value is the OLD one

# Other variables

possibleYears = range(1900, date.today().year + 1)
# The months are listed backwards and in small letters so it's not caps-sensitive, and can compare in terms of index values to see which is earlier
possibleMonths = ["december", "november", "october", "september", "august", "july", "june", "may", "april", "march", "february", "january"]
months = []     # months will contain the months of that particular page where keyWord is found
years = []     # years will contain the heading of the years of that particular page where keyWord is found, [-1] is the oldYear, [-2] is the newYear

# Iterating all pages of PDF to find keyWord
for x in range(reader.numPages):
    pageData = pdf.pages[x]
    text = pageData.extract_text().lower()
#     print(text)
    
    # Checking if the keyword is found in a particular page
    for row in text.split('\n'):
        if row.startswith(keyWord):
            found = 1
            
#             # Since the values of a financial statement is always the last few values
#             # Following example is assuming oldVal is the most right value, and the newVal is the one left of it
#             # Possible like page 9 of sample1.pdf, where there are 4 values instead
#             oldVal = becomeNumber(row.split()[-1])
#             newVal = becomeNumber(row.split()[-2])
#             change = calculateChange(oldVal, newVal)
            
            # Need to get the heading as well to know which values belongs to which year / month
            for word in text.split():
                
                # Trying to get the months
                if word in possibleMonths: #and word not in months:
                    months.append(word)
                
                # To determine which month is the earlier one, assuming there are only 2 months value
                if (len(months) >= 2):
                    
                    # Case 1, when the most right values are the OLD values
                    if (possibleMonths.index(months[-1]) > possibleMonths.index(months[-2])):
                        oldMonth = months[-1]
                        newMonth = months[-2]
                        sameMonth = 0
                        rightNew = -1
                    
                    # Case 2, when the most right values are the NEW values
                    elif (possibleMonths.index(months[-1]) < possibleMonths.index(months[-2])):
                        oldMonth = months[-2]
                        newMonth = months[-1]
                        sameMonth = 0
                        rightNew = 1
                
                    # Case 3, when the months are the same, need to compare the years
                    else:
                        sameMonth = 1
                        rightNew = 0
                
                # Trying to get the years
                try:
                    if int(word) in possibleYears: #and int(word) not in years:
                        years.append(int(word))
                except (ValueError):
                    continue
                    
            # Comparing the years
            
            # Case 1: right values are the NEW values
            if (years[-1] > years[-2]):
                newYear = years[-1]
                oldYear = years[-2]
                rightNew = 1
                sameYear = 0
            
            # Case 2: right values are the OLD values
            elif (years[-1] < years[-2]):
                oldYear = years[-1]
                newYear = years[-2]
                rightNew = -1
                sameYear = 0
            
            # Case 3: both years are the same, need to compare the months
            else:
                sameYear = 1
            
            # Once we know which value is the NEW one, either right or left, based on the variable rightNew
            
            # rightNew == 1 means right value is the NEW one
            if (rightNew == 1):
                newVal = becomeNumber(row.split()[-1])
                oldVal = becomeNumber(row.split()[-2])
                change = calculateChange(oldVal, newVal)
                
            elif (rightNew == -1):
                oldVal = becomeNumber(row.split()[-1])
                newVal = becomeNumber(row.split()[-2])
                change = calculateChange(oldVal, newVal)
            
            # Printing out the drafting respectively based on the different cases
            # [] means need to fill in
            
            if (change < 0):
                if (sameYear == 0 and sameMonth == 0):
                    print("Our {} expenses decreased by {:.2f}% to S$[{}] in the [{}{}], from S$[{}] in the [{}{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
                elif (sameYear == 1):
                    print("Our {} expenses decreased by {:.2f}% to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newMonth, oldVal, oldMonth))
                elif (sameMonth == 1):
                    print("Our {} expenses decreased by {:.2f}% to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newYear, oldVal, oldYear))
            
            elif (change > 0):
                if (sameYear == 0 and sameMonth == 0):
                    print("Our {} expenses increased by {:.2f}% to S$[{}] in the [{}{}], from S$[{}] in the [{}{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
                elif (sameYear == 1):
                    print("Our {} expenses increased by {:.2f}% to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newMonth, oldVal, oldMonth))
                elif (sameMonth == 1):
                    print("Our {} expenses increased by {:.2f}% to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, abs(change), newVal, newYear, oldVal, oldYear))
                    
            else:
                if (sameYear == 0 and sameMonth == 0):
                    print("Our {} expenses stayed the same to S$[{}] in the [{}{}], from S$[{}] in the [{}{}], as a result of [company to provide reason].".format(keyWord, newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
                elif (sameYear == 1):
                    print("Our {} expenses stayed the same to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, newVal, newMonth, oldVal, oldMonth))
                elif (sameMonth == 1):
                    print("Our {} expenses stayed the same to S$[{}] in the [{}], from S$[{}] in the [{}], as a result of [company to provide reason].".format(keyWord, newVal, newYear, oldVal, oldYear))
            
if (found == 0):
    print("Please double check the keyword to look for and try again")
    print(quit)
    quit()
    
# # For error checking
# print("months =")
# print(months)
# print("years =")
# print(years)
# print("found = {}, sameMonth = {}, sameYear = {}, rightNew = {}".format(found, sameMonth, sameYear, rightNew))

Our short-term expenses increased by 21.23% to S$[78310091.0] in the [2020], from S$[64595361.0] in the [2019], as a result of [company to provide reason].
Our short-term expenses decreased by 29.56% to S$[2335630.0] in the [2020], from S$[3315848.0] in the [2019], as a result of [company to provide reason].


ValueError: could not convert string to float: 'value'